# **Hugging Face Embedding 맛보기**



---



- 💡 **NOTE**
    - 이 노트북의 코드를 실행하려면 GPU를 사용하는 것이 좋습니다. 구글 코랩에서는 **런타임 > 런타임 유형 변경 > 하드웨어 가속기 > T4 GPU**를 선택하세요.



---



## **LangChain 간단 소개**

### **LangChain이란?**

- LangChain은 대규모 언어 모델(LLM)을 외부 데이터나 서비스와 쉽게 연결하여, 더 강력하고 실용적인 AI 애플리케이션을 만들 수 있도록 돕는 개발 프레임워크




### **왜 LangChain이 필요한가?**
- **LLM의 한계 극복**
    - **지식의 한계**: LLM은 훈련 데이터에 포함된 정보만 알고 있습니다. 최신 정보나 특정 회사 내부 문서와 같은 비공개 데이터에 대해서는 전혀 알지 못합니다.

    - **상호작용의 한계**: LLM은 스스로 인터넷을 검색하거나, 계산기를 사용하거나, 데이터베이스를 조회하는 등의 외부 작업을 수행할 수 없습니다.

    - **기억력의 한계**: LLM은 기본적으로 상태가 없습니다(stateless). 이전 대화 내용을 기억하지 못하기 때문에, 긴 대화를 나누다 보면 맥락을 잃어버리기 쉽습니다

- LangChain은 마치 **레고(LEGO) 블록**처럼 여러 기능(컴포넌트)을 조립하여 원하는 애플리케이션을 만들 수 있게 해줍니다.


### **LangChain 핵심 요소**
- 💡**체인 (Chains)**
    - LLM 호출을 다른 컴포넌트나 또 다른 LLM 호출과 순차적으로 연결하는 것을 의미
- 💡**RAG (Retrieval-Augmented Generation)**
    - LLM의 '지식의 한계'를 해결하는 가장 대표적인 방법
    - **작동 방식**
        1. **로드 & 분할**: PDF, 웹사이트, DB 등 외부 데이터를 불러와 의미 있는 작은 단위로 쪼갭니다.
        2. **임베딩 & 저장**: 쪼개진 데이터 조각들을 벡터(embedding)로 변환하여 벡터 데이터베이스에 저장합니다.
        3. **검색 & 생성**: 사용자의 질문이 들어오면, 질문과 가장 관련 높은 데이터 조각을 벡터 DB에서 찾아냅니다. 그리고 이 검색된 정보와 원래 질문을 함께 LLM에게 전달하여 정확하고 최신 정보에 기반한 답변을 생성하게 합니다.
- 💡**에이전트 (Agents) & 도구 (Tools)**
    - LLM의 '상호작용의 한계'를 해결
    - 예시: "오늘 서울 날씨를 검색해서 화씨로 알려주고, 그 온도가 32로 나누어 떨어지는지 계산해 줘"라는 요청에 에이전트는 구글 검색 도구와 계산기 도구를 순서대로 사용하여 답변을 찾아냅니다.
- 💡**메모리 (Memory)**
    - LLM의 '기억력의 한계'를 해결
    - 이전 대화의 내용을 저장하고, 필요할 때 LLM에게 다시 전달하여 대화의 맥락을 유지시켜 줍니다. 이를 통해 사용자와 연속적인 대화가 가능한 챗봇을 만들 수 있습니다.



### **LangChain으로 만들 수 있는 것들**
- 💡 **나만의 데이터 챗봇**:
    - 특정 문서(PDF, Notion 등)나 데이터베이스에 대한 질문에 답변하는 챗봇
- 💡 **개인 비서 에이전트**:
    - 웹 검색, 이메일 요약, 스케줄 관리 등을 자율적으로 수행하는 AI 비서
- 💡 **코드 분석 및 생성기**:
    - 복잡한 코드의 의미를 분석하고, 사용자의 요구에 맞는 코드를 생성하는 도구
- 💡 **데이터 분석가**:
    - 자연어 질문을 SQL 쿼리로 변환하여 데이터베이스에서 원하는 정보를 추출하고 분석하는 시스템



---



## **(비용 없이) HuggingFace 임베딩 사용하기**

### **1.라이브러리 설치**

In [ ]:
# 설치
%pip install -q "sentence-transformers>=3.0.0" "langchain-community" "chromadb==1.0.21" "requests==2.32.4"

In [ ]:
!pip install langchain tiktoken openai pypdf  langchain_openai

### **2.라이브러리 불러오기**

In [ ]:
from langchain.vectorstores import Chroma
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.chains import RetrievalQA
import urllib.request
from langchain.document_loaders import PyPDFLoader

### **3.임베딩할 문서 불러오기**

In [ ]:
# 1.다운로드
# !wget -O "2020_경제금융용어 700선_게시.pdf" "https://www.bok.or.kr/portal/cmmn/file/fileDown.do?menuNo=200765&atchFileId=KO_00000000000142606&fileSn=1"
urllib.request.urlretrieve("https://github.com/chatgpt-kr/openai-api-tutorial/raw/main/ch07/2020_%EA%B2%BD%EC%A0%9C%EA%B8%88%EC%9C%B5%EC%9A%A9%EC%96%B4%20700%EC%84%A0_%EA%B2%8C%EC%8B%9C.pdf", filename="2020_경제금융용어 700선_게시.pdf")

('2020_경제금융용어 700선_게시.pdf', <http.client.HTTPMessage at 0x7bca56693ce0>)

In [ ]:
# 2.PDF 파일 읽기 (/content/2020_경제금융용어 700선_게시.pdf)
loader = PyPDFLoader("./2020_경제금융용어 700선_게시.pdf")
texts = loader.load_and_split()     # 페이지별로 쪼갠다.

### **4.문서 살펴보기**

In [ ]:
# 문서의 개수(페이지 수)
print('문서의 수 :', len(texts))

# 공백만 있는 페이지 등은 제외시킨다.
# 실질적으로 살아남는 페이지는 366page
# PDF 문서가 총 366개의 텍스트로 분할됨

문서의 수 : 366


In [ ]:
# 경제 용어가 없는 부분 제거(머릿말 등)
texts = texts[13:]   # 필요한 부분만 잘라내
print('줄어든 texts의 길이 :', len(texts))

줄어든 texts의 길이 : 353


In [ ]:
# 경제 용어가 없는 부분 제거(맺음말: 마지막 데이터 제거)
texts = texts[:-1]
print('마지막 데이터 제거 후 texts의 길이 :', len(texts))

마지막 데이터 제거 후 texts의 길이 : 352


### **5.임베딩하기(임베딩 후 벡터DB에 저장하기)**
1. **임베딩할 모델지정**
    - **intfloat/multilingual-e5-base**
        - Microsoft, 2022년 10월
        - 100개 이상의 언어를 지원하는 고성능 다국어 임베딩 모델로, 텍스트를 의미적으로 정확한 벡터로 변환하여 의미 검색(semantic search) 및 유사도 측정에 주로 사용
2. **임베딩 후 벡터DB에 저장하기(ChromaDB)**
- 💡 **주의** 임베딩할 문서의 양에 따라 시간이 오래 걸릴 수 있습니다.

In [ ]:
# 1) 임베딩: 멀티링구얼 E5 (한/영 모두 양호)
from langchain_community.embeddings import HuggingFaceEmbeddings

embedding = HuggingFaceEmbeddings(
    model_name="intfloat/multilingual-e5-base",
    encode_kwargs={"normalize_embeddings": True}  # 코사인 유사도 안정화
)

# 2) 벡터DB 생성/저장
from langchain_community.vectorstores import Chroma

persist_dir = "/content/chroma_db"
vectordb = Chroma.from_documents(
    documents=texts,                 # LangChain Document 리스트
    embedding=embedding,                   # 또는 embedding_function=embedding (버전에 따라)
    persist_directory=persist_dir
)
vectordb.persist()  # 임베딩을 디스크에 저장


/tmp/ipython-input-1502505462.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

/tmp/ipython-input-1502505462.py:18: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()  # 임베딩을 디스크에 저장


### **6.질의 테스트**

In [ ]:
# 3) 질의 테스트
#   "k": 3 --> 가장 관련성이 높은 상위 3개의 문서(또는 청크)만 가져오도록 지정
retriever = vectordb.as_retriever(search_kwargs={"k": 3})
docs = retriever.get_relevant_documents("J커브란")  # 요약해줘: 파인콘과 밀버스 차이
print([d.page_content[:120] for d in docs])


/tmp/ipython-input-1602913768.py:4: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents("J커브란")  # 요약해줘: 파인콘과 밀버스 차이


['346\n경제금융용어 700선\nIMF 포지션\n국제통화기금(IMF) 회원국 쿼타에서 IMF가 보유하고 있는 해당 회원국 통화를 차감한 \n부분을 말하며 IMF 리저브포지션(Reserve Position) 또는 리저브트란셰', '157\nㅅ \n영향을 미쳐 성장을 제약할 수 있다는 주장이 이론적･실증적인 연구에 의해 힘을 얻고 \n있다. 이를 배경으로 포스트케인지안들은 노동소득분배율 변화가 경제성장과 정(+)의 \n관계를 가지면 임금주도경제(wag', '97\nㄷ \n제가 시행됨에 따라 은행들이 금융기관을 대상으로 CD를 발행하는 경우 대부분 CD를 \n실물발행하지 않고 한국예탁결제원을 통해 등록발행하고 있다 . \n디레버리징\n디레버리징(deleveraging)은 부채를 ']


In [ ]:
for d in docs:
  print(d)
  print('-' * 50)

page_content='346
경제금융용어 700선
IMF 포지션
국제통화기금(IMF) 회원국 쿼타에서 IMF가 보유하고 있는 해당 회원국 통화를 차감한 
부분을 말하며 IMF 리저브포지션(Reserve Position) 또는 리저브트란셰 (Reserve 
Tranche)라고도 한다. IMF 포지션 인출은 국제수지상의 필요성이 전제되기는 하지만 
IMF로부터의 여타 신용인출과 달리 IMF가 인출조건을 부과하지 않기 때문에 회원국은 
자국의 국제수지 사정에 따라 사실상 자유롭게 인출할 수 있다. 이에 따라 IMF 포지션은 
회원국의 입장에서는 언제든지 인출이 가능한 청구권이므로 해당국의 외환보유액에 
포함된다. 회원국이 IMF 포지션을 인출하는 경우에는 그에 상응하는 자국통화를 납입하
여야 하며 그만큼 IMF 포지션은 줄어들게 된다. IMF 포지션은 IMF 재원의 일종이므로 
IMF는 회원국별 리저브포지션의 일정 부분에 대하여 재원조달 수수료에 해당하는 보상
금을 지급하고 있다 . 
 연관검색어 : 국제통화기금(IMF), 특별인출권 (SDR)
J커브효과
이론적으로 환율이 상승할 경우 수출은 늘어나고 수입은 줄어들어 경상수지가 개선된
다. 그러나 현실에서는 초기에 경상수지가 악화되다가 어느 정도의 시간이 지난 후에야 
경상수지가 개선되는 효과가 나타나는데, 이를 J커브효과라고 한다. 이는 환율의 상승으
로 경상수지가 실질적으로 개선되기까지는 어느 정도의 시간이 소요되기 때문이다 . 
환율 상승으로 국내 수출품 가격이 하락하더라도 이러한 가격 하락에 대응하여 수출물량
이 증가하기까지는 시간이 소요될 수밖에 없다. 이에 따라 단기적으로는 수출가격에 
물량을 곱한 수출금액이 오히려 감소하면서 경상수지가 악화될 수 있다 .
 연관검색어 : 수출입물가지수
KIKO
환율이 특정 구간(barrier)에 도달하는 경우 옵션이 발효(KI; Knock-In)되거나 소멸
(KO; Knock-Out)되는 조건이 부과된 비정형적인 통화옵션 거래의 일종이다. 수출기업
의 경우 옵션기간 



---



## **[미션]** : HuggingFace 임베딩 테스트하기

앞의 내용을 참고하여 새로운 pdf 파일을 준비한 후 Hugging Face Embedding 기능을 테스트해보세요.

In [ ]:
import os, urllib.request
from langchain.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# ---------- 0) 설정 ----------
PDF_URL = "https://arxiv.org/pdf/1801.04293.pdf"
PDF_NAME = "critical_success_factors_gamedev.pdf"
PERSIST_DIR = "./chroma_db_csf_gamedev"
TOP_K = 3
THRESHOLD = 0.55  # 신뢰도 경고 기준

# ---------- 1) PDF 다운로드 ----------
print("[1/6] PDF 다운로드 중 ...")
urllib.request.urlretrieve(PDF_URL, filename=PDF_NAME)
print(f"  → saved: {os.path.abspath(PDF_NAME)}")

# ---------- 2) PDF 로드 & 청크 분할 ----------
print("[2/6] PDF 로드 및 텍스트 분할 ...")
loader = PyPDFLoader(PDF_NAME)
pages = loader.load()  # 페이지 단위 로드

# 논문은 한 페이지가 길어 검색 성능을 위해 청크(문자열 길이)로 다시 분할
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,   # 청크 길이
    chunk_overlap=120, # 겹침
    separators=["\n\n", "\n", ". ", " "]
)
docs = splitter.split_documents(pages)
print("  원본 페이지 수   :", len(pages))
print("  청크된 문서 수   :", len(docs))

# e5 권장: 문서에는 'passage:' 프리픽스 부여
for d in docs:
    d.page_content = "passage: " + d.page_content

# ---------- 3) 임베딩(e5) ----------
print("[3/6] 임베딩 모델 로드 (intfloat/multilingual-e5-base, normalize=True) ...")
embedding = HuggingFaceEmbeddings(
    model_name="intfloat/multilingual-e5-base",
    encode_kwargs={"normalize_embeddings": True}
)

# ---------- 4) Chroma 벡터DB 구축/저장 ----------
print("[4/6] Chroma 벡터DB 생성/저장 ...")
vectordb = Chroma.from_documents(
    documents=docs,
    embedding=embedding,
    persist_directory=PERSIST_DIR
)
vectordb.persist()
print(f"  → persisted dir: {os.path.abspath(PERSIST_DIR)}")

# ---------- 5) 질의 함수 ----------
def ask(query_text, top_k=TOP_K, threshold=THRESHOLD):
    """
    e5는 query/passage 프리픽스를 지키면 검색 품질이 좋아집니다.
    - query_text: 사용자 질문
    - top_k: 상위 검색 문서 수
    - threshold: 유사도 경고 기준
    """
    q = "query: " + query_text
    retriever = vectordb.as_retriever(search_kwargs={"k": top_k})
    results = retriever.get_relevant_documents(q)

    print(f"\n✅ 질문: {query_text}")
    print("="*70)
    # 스니펫 미리보기
    for i, d in enumerate(results, 1):
        meta = d.metadata
        page = meta.get("page", "?")
        snippet = d.page_content[:300].replace("\n", " ")
        print(f"[{i}] page={page} | snippet: {snippet} ...")

    # 유사도 점수(재계산): 쿼리 vs 각 결과 청크
    q_emb = embedding.embed_query(q)  # 정규화됨
    doc_embs = embedding.embed_documents([d.page_content for d in results])
    sims = cosine_similarity([q_emb], doc_embs)[0]

    print("\n상위 k개 유사도:")
    for i, s in enumerate(sims, 1):
        print(f"  [{i}] sim={s:.4f}")

    best_i = int(np.argmax(sims))
    best_sim = float(sims[best_i])
    best_doc = results[best_i]

    print(f"\n★ 최종 선택: page={best_doc.metadata.get('page','?')} (sim={best_sim:.4f})")
    if best_sim < threshold:
        print("⚠️ 유사도가 낮습니다. 질문을 더 구체화하거나 청크/전처리를 조정해 보세요.")
    return best_doc, best_sim

# ---------- 6) 예시 질의 ----------
queries = [
    "게임 개발 성공 요인은 무엇인가?",                      # 핵심 성공 요인 개요
    "팀 커뮤니케이션이 개발 성과에 미치는 영향은?",          # 커뮤니케이션/협업
    "테스트와 품질보증(QA)의 중요성은?",                   # 테스트/QA
    "요구사항 변경 관리가 프로젝트 품질에 주는 영향은?",     # 범위/요구사항 관리
    "개발 도구와 파이프라인 관련 권장 사항은?"              # 도구/프로세스
]

for q in queries:
    _doc, _sim = ask(q, top_k=3, threshold=THRESHOLD)

print("\n[완료] 논문 기반 임베딩 검색 테스트가 끝났습니다!")


[1/6] PDF 다운로드 중 ...
  → saved: /content/critical_success_factors_gamedev.pdf
[2/6] PDF 로드 및 텍스트 분할 ...
  원본 페이지 수   : 51
  청크된 문서 수   : 163
[3/6] 임베딩 모델 로드 (intfloat/multilingual-e5-base, normalize=True) ...
[4/6] Chroma 벡터DB 생성/저장 ...
  → persisted dir: /content/chroma_db_csf_gamedev

✅ 질문: 게임 개발 성공 요인은 무엇인가?
[1] page=4 | snippet: passage: opment.  Nevertheless, the differences between  software engineering and games development are  not exclusive; it seems that traditional software  development does not fully support game devel- opment activities and provide process assessment  procedures [28]. So, we need key success factor ...
[2] page=5 | snippet: passage: Journal of Computer Science and Technology, 31(5):925-950, DOI: 10.007/s11390-016-1673-z, Springer, September 2016.  - 6 -  ment processes have different phases and are in- fluenced by many factors. Identifying the key suc- cess factors in a game development process is ex- tremely important ...
[3] page=5 | snippet: passage: in